## Airbnb Dataset: 
https://www.kaggle.com/airbnb/seattle/data
### Context
Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in Beijing, China.

### Content
The following Airbnb activity is included in this Beijing dataset:

Listings, including full descriptions and average review score
Reviews, including unique id for each reviewer and detailed comments
Calendar, including listing id and the price and availability for that day

### Inspiration
Can you describe the vibe of each Seattle neighborhood using listing descriptions?
What are the busiest times of the year to visit Seattle? By how much do prices spike?
Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle?
For more ideas, visualizations of all Seattle datasets can be found here.

### Acknowledgement
This dataset is part of Airbnb Inside, and the original source can be found here.
(http://insideairbnb.com/get-the-data.html)


## Analysis Procedures (CRISP-DM)


### 1. Business understanding

    - Can you build a recommendation model, by grouping similar listings into one cluster?
    - What are the busiest times of the year to visit Seattle?
    - How the price changes over one year?
    - Can you predict/suggest a price given a listing information? which features impact the price most?
    - Can you predict the review score? What features makes the customers feel more satisfied?
    - * Can you predict the occupancy rate?

    
### 2. Data understanding
    - Data exploration
    
### 3. Data preparation
    - Handling missing values, categorical values, and feature engineering.

### 4. Modelling
    - Use clustering algorithms for grouping/recommendation purpose.
    - Use regression to suggest owner the reseaonable price range.

### 5. Results
    - Answer questions in step 1 'Business understanding' with data visualization.

## 2. Data understanding

In this part, the three datasets in csv format is imported into dataframes using `pandas`, and explored using available pandas functions in below cells. The information obtained about the datasets is summarized as below in this cell.

**Calender dataset** contains the price information for each listing in a calender year. There are 3818 unique `listing_id` in the dataset, and for each `listing_id` there are 365 rows of price corresponding one day between _2016-01-04 and 2017-01-02_ .<br>
The `available` column has two unique values _'t' or 'f'_ meaning _True or False_. When a listing is not available for the day, the columns `price` is _nan_.

**Listing dataset** contains the full description for each listing scraped on _2016-01-04_ , with 3818 rows describing 3818 unique listings. <br>
There are 92 columns/features in this dataset, thus a lot of information that need to be selectively used in later session. <br>
Telling from the column names, the features can be roughly divided into below categories: listing info, host info, location, room/house info, price and booking, reviews, and policies.

**Reviews dataset** contains all the review entries for above mention 3818 listings by _2016-01-03_ since 2009. Each row records the review's info and the detailed text comments for a listing in a certain day, without a numerical score feature. <br>
The positivity of each comments could be predicted through a NLP modelling, but it's not in the objectives of this analysis and the _listing_ dataset already have features of reviews in numerical scores. Thus, the _reviews_ dataset will not be used for furthur analysis.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

calender_path = '/Users/clairegong/Desktop/UdacityDataScienceNanoDegree/seattle airbnb dataset/calendar.csv'
listing_path = '/Users/clairegong/Desktop/UdacityDataScienceNanoDegree/seattle airbnb dataset/listings.csv'
reviews_path = '/Users/clairegong/Desktop/UdacityDataScienceNanoDegree/seattle airbnb dataset/reviews.csv'

calender=pd.read_csv(calender_path)
listing=pd.read_csv(listing_path)
reviews=pd.read_csv(reviews_path)


In [57]:
calender.head(5)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [56]:
calender.listing_id.nunique()

3818

In [41]:
calender.listing_id.value_counts().head()

6752031     365
7404370     365
1259305     365
4672934     365
10310373    365
Name: listing_id, dtype: int64

In [35]:
print('Date range of calender data is between {} and {}.'.format(calender.date.min(),calender.date.max()))

Date range of calender data is between 2016-01-04 and 2017-01-02.


In [3]:
calender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null object
price         934542 non-null object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


In [27]:
# listing.describe()

In [3]:
# listing.info() #3818 entries,92 columns

In [60]:
listing.shape

(3818, 92)

In [45]:
listing.columns # 92 columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [54]:
listing.sample()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
3468,6118198,https://www.airbnb.com/rooms/6118198,20160104002432,2016-01-04,Private Suite in Modern Home,"Modern, clean, and light-filled! Private bedro...",All the comforts of home! A comfortable queen-...,"Modern, clean, and light-filled! Private bedro...",none,We're lucky to live in the Alaska junction nei...,...,10.0,f,NaN,WASHINGTON,t,moderate,f,f,1,2.8


In [53]:
# listing.iloc[:,:10].sample()

In [22]:
reviews.sample(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
15313,9460,1510659,2012-06-18,1866904,Katie,I wish I could give 10 stars for Siena's apart...
47300,1956553,32650545,2015-05-19,4482226,Andrew,Mia and Chris made it simple to check-in and c...
28916,5057466,30488346,2015-04-22,335510,Sun,Eli's friendly hospitality and beautiful home ...
15627,20927,2360406,2012-09-21,3125596,Sarah,Absolutely loved my stay at the cottage! Liz a...
40439,6451305,43050050,2015-08-17,10490385,Austin,My experience in Seattle was amazing! I couldn...


In [23]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [39]:
print('Date range of review data is between {} and {}.'.format(reviews.date.min(),reviews.date.max()))

Date range of review data is between 2009-06-07 and 2016-01-03.


## 3. Data preparation

The _listing_ and _calender_ datasets will be processed in this section to be ready for comsumption for prediction models. From the information gathered in last part, below processing is in order: data cleaning, feature engineering, missing values imputing, categorical values imputing etc.



**Calender dataset**

In [13]:
#Data cleaning
# remove dollar signs $ and , in the price feature
calender.price = calender.price.replace('[\$,\,]','', regex=True)
calender.price = pd.to_numeric(calender.price)

# modify available feature to boolean values to be meaningful, and convenient for furthur calculation.
calender.available.replace({'t':True, 'f': False}, inplace=True)
calender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null bool
price         934542 non-null float64
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 33.2+ MB


In [80]:
#Feature engineering
# One of our purposes is to see the price trends, considering neighborhoods, so I need to extract the useful 
# neighbourhood features from the listing dataset. After some checking, neighbourhood_group_cleansed feature 
# is the best one, merge this feature to the calender dataset.
neighourhood_group=listing[['id','neighbourhood_group_cleansed']].\
    rename(columns={'id':'listing_id','neighbourhood_group_cleansed':'neighbourhood'})
calender = calender.merge(neighourhood_group, how='left')

calender.sample(5)

,listing_id,date,available,price,neighbourhood
1080641,7651702,2016-09-01,True,100.0,Capitol Hill
293451,10126050,2016-12-25,False,NaN,University District
749665,23356,2016-11-19,True,240.0,Beacon Hill
500388,23430,2016-12-07,False,NaN,Downtown
1224169,5123904,2016-11-23,False,NaN,Other neighborhoods


### Listing dataset

This dataset has too many(i.e. 92) features, it is known that models trained with too many features are prone to overfit and predict badly on unseen data, thus I need to select those possibly useful in answering our business questions, i.e. predicting prices and scores. Features with descriptive text values is not useful in this context
so I will exclude them all, with the exception for a few with categorical values. Also features that are likely to impact on price and review score but not much, are also discarded.

To select features, the data scientist/analyst need to examine EVERY feature, but a more efficient way I usually do is to examine features by BATCH of 10, or a number of your choice. 

**Examing** includes but is not limited to: checking <br>
- the unique values of categorical values by `nunique` `value_counts`
- the number of missing values by `isnull`
- data distribution of numerical values by `describe`
- comparing features with similar names by `filter`
- and so on


**Below are some features that I discarded as they will provide little information on predicting prices and review scores. Accompanying the feature names are some thoughts written for readers' reference.** <br>
- _experiences_offered_ : 3055/3055 is 'none'
- _host_acceptance_rate_ : 3044/3045 is 100%
- _host_total_listings_count_ : has same info as host_listings_count
- _host_verifications_ : could be useful, by splitting list into elements then into categorical values such as 'has_email', but is not worth the effort in this case.
- _host_has_profile_pic_ : 3809/3817 True
- _square_feet_ : is 97.4% missing using #listing.square_feet.isnull().mean()
- _weekly_price_ : is 47.3% missing, it could be organized to a no_weekly_price feature, but will be discarded in this context.
- _monthly_price_ : is 60.3% missing, reason to discard is as above.
- _calendar_updated_ : values could be unified into number of days ago, but it will be discarded in this context.
- _has_availability_ : 100% True
- _availability_30_ : and it stands for how many days the room is available in the next 30 days. availability_num features follows the same logic. Listing dataset will be used to predict price and score, so these features I think are less significant. To calculate occupancy rate, calender dataset can be used instead.
- _review_scores_xx_ : features I only keep one feature: review_scores_rating as it is representive of overall experience.
- _requires_license_ : 100% False
- _require_guest_profile_picture_ : 3497/3828 False, this could be useful but considered not significant, thus is discarded.
- _require_guest_phone_verification_ : reason to discard is similar to above.

**Codes below are the typical ones I used to examine the feature.**

In [81]:
listing.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [194]:
# Explore "score" features
scores=listing.filter(like='score')
# I tried to sum the scores in scale of 10, but seems not equals to the 100-scale review_scores_rating.
scores['scores_sum']=listing.filter(like='score').iloc[:,1:].sum(axis=1)
scores[['review_scores_rating','scores_sum']].head(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,review_scores_rating,scores_sum
0,95.0,59.0
1,96.0,60.0
2,97.0,60.0
3,NaN,0.0
4,92.0,56.0


In [205]:
listing.reviews_per_month.describe()

count    3191.000000
mean        2.078919
std         1.822348
min         0.020000
25%         0.695000
50%         1.540000
75%         3.000000
max        12.150000
Name: reviews_per_month, dtype: float64

In [206]:
listing.require_guest_profile_picture.value_counts()#.sort_index()

1     2451
2      575
3      262
4      130
5       70
6       65
37      37
31      36
34      34
11      22
21      21
7       21
9       19
8       19
18      18
15      16
12      12
10      10
Name: calculated_host_listings_count, dtype: int64

In [204]:
# Codes for examining features by batch of 10 
listing.iloc[:,0:10].sample()
listing.iloc[:,10:20].sample()
listing.iloc[:,20:30].sample()
listing.iloc[:,30:40].sample()
listing.iloc[:,40:50].sample()
listing.iloc[:,50:60].sample()
listing.iloc[:,60:70].sample()
listing.iloc[:,70:80].sample()
listing.iloc[:,80:90].sample()
listing.iloc[:,90:100].sample()

,calculated_host_listings_count,reviews_per_month
1967,1,0.84


### Finally! I have the features I need

In [219]:

features = ['id','host_since','host_response_time','host_response_rate','host_is_superhost','host_listings_count',\
            'host_identity_verified','neighbourhood_group_cleansed','is_location_exact','property_type','room_type',\
           'accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit',\
            'cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews',\
           'review_scores_rating','instant_bookable','cancellation_policy','reviews_per_month']

listing_cleansed=listing[features]
listing_cleansed.head()

### but ... there is much more to be done

In [221]:
# Count of amenties provided is made to a new feature for ease of anaylis.
amenities=listing.amenities.str.split(',')
listing_cleansed['amenities']=amenities.apply(lambda row: len(list(row)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [226]:
listing_cleansed[['security_deposit','cleaning_fee']]=\
    listing_cleansed[['security_deposit','cleaning_fee']].fillna(0.0)

In [225]:
listing_cleansed.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_group_cleansed,is_location_exact,property_type,...,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy,reviews_per_month
0,241032,2011-08-11,within a few hours,96%,f,3.0,t,Queen Anne,t,Apartment,...,0,2,$5.00,1,365,207,95.0,f,moderate,4.07
1,953595,2013-02-21,within an hour,98%,t,6.0,t,Queen Anne,t,Apartment,...,$40.00,1,$0.00,2,90,43,96.0,f,strict,1.48
2,3308979,2014-06-12,within a few hours,67%,f,2.0,t,Queen Anne,t,House,...,$300.00,10,$25.00,4,30,20,97.0,f,strict,1.15
3,7421966,2013-11-06,NaN,NaN,f,1.0,t,Queen Anne,t,Apartment,...,0,1,$0.00,1,1125,0,NaN,f,flexible,NaN
4,278830,2011-11-29,within an hour,100%,f,2.0,t,Queen Anne,t,House,...,$125.00,6,$15.00,1,1125,38,92.0,f,strict,0.89
